In [11]:
import pybamm as pb;import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;#import imageio
from scipy.io import savemat,loadmat;from pybamm import constants,exp;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
import openpyxl
import traceback
import timeit
import multiprocessing
import scipy.optimize
import random;import time, signal
import sys  
str_path_0 = os.path.abspath(os.path.join(pb.__path__[0],'..'))
str_path_1 = os.path.abspath(os.path.join(str_path_0,"wip\Rio_Code\Fun_P2"))
sys.path.append(str_path_1) 
from Fun_P2 import * 

In [12]:
model = pb.lithium_ion.DFN()
param = pb.ParameterValues("OKane2023")
# Adjust SEI parameters
param.update({"Inner SEI lithium interstitial diffusivity [m2.s-1]": 5e-19})
# Override SEI thickness dependence of lithium plating
param.update({"Dead lithium decay rate [s-1]": 1e-6})
param.update({"Lithium plating kinetic rate constant [m.s-1]": 1e-10})
# Adjust mechanical degradation parameters
param.update({"Negative electrode cracking rate": 1e-22})
param.update({"Negative electrode LAM constant proportional term [s-1]": 5e-8})
param.update({"Positive electrode LAM constant proportional term [s-1]": 1e-8})
param.update({"Electrolyte conductivity [S.m-1]": electrolyte_conductivity_EC_EMC_3_7_Landesfeind2019_Constant})
param.update({"Electrolyte diffusivity [m2.s-1]": electrolyte_diffusivity_EC_EMC_3_7_Landesfeind2019_Constant})
param.update({"1 + dlnf/dlnc": electrolyte_TDF_EC_EMC_3_7_Landesfeind2019_Constant})
param.update({"Cation transference number": electrolyte_transference_number_EC_EMC_3_7_Landesfeind2019_Constant})


In [14]:
# define experiment
V_max = 4.2;        V_min = 2.5; 
discharge_time_mins = 0.15* 60 * 4.86491/5
charge_time_mins = 0.5* 60 * 4.86491/5
exp_RPT_0p1C_text = [ ( 
    f"Charge at 0.3C until {V_max} V",  
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (5 minute period)",  
    f"Discharge at 0.1C until {V_min} V (10 minute period)",  
    "Rest for 1 hours (5 minute period)",  
    f"Charge at 0.1C until {V_max} V (10 minute period)",
    ) ] # *1
exp_RPT_refill_text = [ (
    f"Charge at 0.3C until {V_max} V",  
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours (5 minute period)",  
    ) ] # * 1
exp_RPT_GITT_text = [ (
    "Rest for 58 seconds (29 second period)",  
    "Rest for 2 seconds (0.1 second period)",  
    f"Discharge at C/2 for 2 seconds or until {V_min}V (0.1 second period)",
    f"Discharge at C/2 for 286 seconds or until {V_min}V",
    "Rest for 1 hour (5 minute period)",  
    ) ] # * 24
exp_preAge_text = [ (    
    f"Discharge at 1C for {discharge_time_mins} minutes or until {V_min}V", 
    "Rest for 3 hours (15 minute period)",
    ) ] # * 1
Experiment_0p1C = pb.Experiment( exp_RPT_0p1C_text * 1     )
Experiment_refill = pb.Experiment( exp_RPT_refill_text * 1     )
Experiment_GITT = pb.Experiment( exp_RPT_GITT_text * 24     )
Experiment_preAge = pb.Experiment( exp_preAge_text * 1     )
Exps = [Experiment_0p1C,Experiment_refill,Experiment_GITT,Experiment_preAge]
Texts = ["0.1C","Refill","GITT","Pre Age"]

In [15]:
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 30,  # negative particle
    "r_p": 20,  # positive particle
}

In [19]:
# the following scripts show that if solve independetly, it is very quick! (25s )
Sols = []
TotTimeer = pb.Timer() 
for exp,text in zip(Exps,Texts):
    start_1 = pb.Timer()   # timeit.default_timer()
    sim1 = pb.Simulation(
        model, 
        experiment=exp,
        parameter_values=param,
        solver=pb.CasadiSolver(),
        var_pts=var_pts,
    )
    Sols.append(sim1.solve())

    # stop_1 = timeit.default_timer()
    print(f'running time: for Exp {text} is {start_1.time()}')
print(f'running time: for All is {TotTimeer.time()}')

running time: for Exp 0.1C is 8.003 s
running time: for Exp Refill is 2.700 s
running time: for Exp GITT is 12.767 s
running time: for Exp Pre Age is 2.297 s
running time: for All is 25.768 s


In [17]:
# then what if need to build based on previous solution? - still pretty quick!

Sols = []; Models = [];
for i in range(4):  #  in zip(Exps,Texts):
    if i==0:
        start_1 = timeit.default_timer()
        model = pb.lithium_ion.DFN()
        sim1 = pb.Simulation(
            model, 
            experiment=Exps[i],
            parameter_values=param,
            solver=pb.CasadiSolver(),
            var_pts=var_pts,
        )
        Sols.append(sim1.solve())
        Models.append(model)
        stop_1 = timeit.default_timer()
        print(f'running time: for Exp {Texts[i]} is {str(stop_1 - start_1)} s')
    else:
        start_1 = timeit.default_timer()
        Model_new = Models[-1].set_initial_conditions_from(Sols[-1], inplace=False)
        sim1 = pb.Simulation(
            Model_new, 
            experiment=Exps[i],
            parameter_values=param,
            solver=pb.CasadiSolver(),
            var_pts=var_pts,
        )
        Sols.append(sim1.solve())
        Models.append(Model_new)

        stop_1 = timeit.default_timer()
        print(f'running time: for Exp {Texts[i]} is {str(stop_1 - start_1)} s')


running time: for Exp 0.1C is 7.171382499999936 s
running time: for Exp Refill is 2.4498528999999962 s
running time: for Exp GITT is 13.01265669999998 s


At t = 0.00371461 and h = 2.98094e-019, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.00371461 and h = 4.81095e-019, the corrector convergence failed repeatedly or with |h| = hmin.


running time: for Exp Pre Age is 2.3340688000000682 s


In [18]:
# add a bit ageing?
# then what if need to build based on previous solution? - still pretty quick!

Sols = []; Models = [];
for i in range(4):  #  in zip(Exps,Texts):
    if i==0:
        start_1 = timeit.default_timer()
        model = pb.lithium_ion.DFN(options={
            "lithium plating": "partially reversible",
            "contact resistance": "true",
            })
        sim1 = pb.Simulation(
            model, 
            experiment=Exps[i],
            parameter_values=param,
            solver=pb.CasadiSolver(),
            var_pts=var_pts,
        )
        Sols.append(sim1.solve())
        Models.append(model)
        stop_1 = timeit.default_timer()
        print(f'running time: for Exp {Texts[i]} is {str(stop_1 - start_1)} s')
    else:
        start_1 = timeit.default_timer()
        Model_new = Models[-1].set_initial_conditions_from(Sols[-1], inplace=False)
        sim1 = pb.Simulation(
            Model_new, 
            experiment=Exps[i],
            parameter_values=param,
            solver=pb.CasadiSolver(),
            var_pts=var_pts,
        )
        Sols.append(sim1.solve())
        Models.append(Model_new)

        stop_1 = timeit.default_timer()
        print(f'running time: for Exp {Texts[i]} is {str(stop_1 - start_1)} s')


running time: for Exp 0.1C is 7.895964399999912 s
running time: for Exp Refill is 2.7453550999998697 s
running time: for Exp GITT is 13.26060069999994 s


At t = 0.00382587 and h = 4.80267e-016, the corrector convergence failed repeatedly or with |h| = hmin.
At t = 0.00382587 and h = 2.46462e-016, the corrector convergence failed repeatedly or with |h| = hmin.


running time: for Exp Pre Age is 2.3426165000000765 s
